### Importing road augmentation library Automold and helper functions library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/sensing_project/snow_test')
import Automold as am
import Helpers as hp

# data augumentation

light:

*   size: (1,2)
*   num_snowflakes: 1500

moderate:
*   size: (1,4)
*   num_snowflakes: 2500

heavy:
*   size: (1,6)
*   num_snowflakes: 3000

severe:
*   size: (1,8)
*   num_snowflakes: 4000

## addsnow (with snow cover)

In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
# Import Automold and Helpers modules
sys.path.append('/content/drive/MyDrive/sensing_project/snow_test')
import Automold as am
import Helpers as hp

def hexagon_star_vertices(center, size):
    """ 
    Generate vertices of a hexagon or hexagram.
    center: The center point coordinates (x, y).
    size: The size of the hexagon/hexagram.
    """
    x, y = center
    vertices = []
    for i in range(6):
        angle = np.radians(i * 60)
        vertex_x = int(x + size * np.cos(angle))
        vertex_y = int(y + size * np.sin(angle))
        vertices.append([vertex_x, vertex_y])
    return np.array(vertices, np.int32).reshape((-1, 1, 2))

def generate_snow(image):
    def add_snowflakes(snowy_images):
        blended_images = []

        for original_image in snowy_images:
            height, width, _ = original_image.shape

            # Create a black canvas of the same size as the original image
            snow_image = np.zeros_like(original_image, dtype=np.uint8)

            # Total number of snowflakes
            num_snowflakes = 4000

            # Divide into upper and lower regions
            num_snowflakes_upper = int(num_snowflakes * 0.8)  # 80% of the snowflakes in the upper region
            num_snowflakes_lower = int(num_snowflakes * 0.2 * 0.6)  # 20% in the lower region, 60% of upper region count

            # Generate snowflakes for the upper region
            for _ in range(num_snowflakes_upper):
                x = np.random.randint(0, width)
                y = np.random.randint(0, int(height * 0.8))
                size = np.random.randint(1, 8)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), size)
                intensity = np.random.randint(250, 256)
                cv2.fillPoly(snow_image, [hexagon_star_vertices_array], (intensity, intensity, intensity))

            # Generate snowflakes for the lower region
            for _ in range(num_snowflakes_lower):
                x = np.random.randint(0, width)
                y = np.random.randint(int(height * 0.8), height)
                size = np.random.randint(1, 8)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), size)
                intensity = np.random.randint(250, 256)
                cv2.fillPoly(snow_image, [hexagon_star_vertices_array], (intensity, intensity, intensity))

            # Apply Gaussian blur to the snowflake image
            blurred_snow_image = cv2.GaussianBlur(snow_image, (15, 15), 0)

            # Logic for generating new snowflakes (select 10% of the original snowflakes)
            original_hexagon_indices = random.sample(range(num_snowflakes_upper + num_snowflakes_lower), 
                                                    int(0.1 * (num_snowflakes_upper + num_snowflakes_lower)))

            for idx in original_hexagon_indices:
                if idx < num_snowflakes_upper:
                    x = np.random.randint(0, width)
                    y = np.random.randint(0, int(height * 0.8))
                else:
                    x = np.random.randint(0, width)
                    y = np.random.randint(int(height * 0.8), height)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), 1)
                
                for _ in range(6):
                    if _ == 0:
                        center_x, center_y = random.choice(hexagon_star_vertices_array)
                    else:
                        center_x, center_y = random.choice(new_hexagon_vertices)

                    new_size = np.random.randint(1, 3)
                    new_hexagon_vertices = hexagon_star_vertices((center_x, center_y), new_size)
                    cv2.fillPoly(snow_image, [new_hexagon_vertices], (255, 255, 255))

            # Blend the original image with the snowflake layer
            blended_image = cv2.addWeighted(original_image, 1, blurred_snow_image, 0.7, 0)
            blended_images.append(blended_image)

        return blended_images

    # Call Automold library to add snow effect
    snowy_image = am.add_snow([image], snow_coeff=0.7)[0]

    # Call add_snowflakes function to add snowflakes
    final_image = add_snowflakes(snowy_image)

    return final_image

# Set input and output folder paths
input_folder_path = '/content/drive/MyDrive/sensing_project/data/row_image/severe'
output_folder_path = '/content/drive/MyDrive/sensing_project/data/augumented_allsnow_2/severe'

# Get all files in the input folder
input_files = os.listdir(input_folder_path)

# Iterate through each file
for input_file_name in input_files:
    input_file_path = os.path.join(input_folder_path, input_file_name)

    # Read the image
    original_image = cv2.imread(input_file_path)

    # Process the image
    blended_image = generate_snow(original_image)

    # Generate the output file path
    output_file_path = os.path.join(output_folder_path, input_file_name)

    # Save the processed image
    cv2.imwrite(output_file_path, blended_image)

print("Processing complete, images saved to", output_folder_path)


处理完成，图像已保存到 /content/drive/MyDrive/sensing_project/data/augumented_allsnow_2/severe


## snowfall

In [ ]:
import os
import cv2
import numpy as np

def hexagon_star_vertices(center, size):
    """ 
    Generate vertices of a hexagon or hexagram.
    center: The center point coordinates (x, y).
    size: The size of the hexagon/hexagram.
    """
    x, y = center
    vertices = []
    for i in range(6):
        angle = np.radians(i * 60)
        vertex_x = int(x + size * np.cos(angle))
        vertex_y = int(y + size * np.sin(angle))
        vertices.append([vertex_x, vertex_y])
    return np.array(vertices, np.int32).reshape((-1, 1, 2))

def generate_snow(image):
    def add_snowflakes(snowy_images):
        blended_images = []

        for original_image in snowy_images:
            height, width, _ = original_image.shape

            # Create a black canvas of the same size as the original image
            snow_image = np.zeros_like(original_image, dtype=np.uint8)

            # Total number of snowflakes
            num_snowflakes = 4000

            # Divide into upper and lower regions
            num_snowflakes_upper = int(num_snowflakes * 0.8)  # 80% of the snowflakes in the upper region
            num_snowflakes_lower = int(num_snowflakes * 0.2 * 0.6)  # 20% in the lower region, 60% of upper region count

            # Generate snowflakes for the upper region
            for _ in range(num_snowflakes_upper):
                x = np.random.randint(0, width)
                y = np.random.randint(0, int(height * 0.8))
                size = np.random.randint(1, 8)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), size)
                intensity = np.random.randint(250, 256)
                cv2.fillPoly(snow_image, [hexagon_star_vertices_array], (intensity, intensity, intensity))

            # Generate snowflakes for the lower region
            for _ in range(num_snowflakes_lower):
                x = np.random.randint(0, width)
                y = np.random.randint(int(height * 0.8), height)
                size = np.random.randint(1, 8)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), size)
                intensity = np.random.randint(250, 256)
                cv2.fillPoly(snow_image, [hexagon_star_vertices_array], (intensity, intensity, intensity))

            # Apply Gaussian blur to the snowflake image
            blurred_snow_image = cv2.GaussianBlur(snow_image, (15, 15), 0)

            # Logic for generating new snowflakes (select 10% of the original snowflakes)
            original_hexagon_indices = random.sample(range(num_snowflakes_upper + num_snowflakes_lower), 
                                                    int(0.1 * (num_snowflakes_upper + num_snowflakes_lower)))

            for idx in original_hexagon_indices:
                if idx < num_snowflakes_upper:
                    x = np.random.randint(0, width)
                    y = np.random.randint(0, int(height * 0.8))
                else:
                    x = np.random.randint(0, width)
                    y = np.random.randint(int(height * 0.8), height)
                hexagon_star_vertices_array = hexagon_star_vertices((x, y), 1)
                
                for _ in range(6):
                    if _ == 0:
                        center_x, center_y = random.choice(hexagon_star_vertices_array)
                    else:
                        center_x, center_y = random.choice(new_hexagon_vertices)

                    new_size = np.random.randint(1, 3)
                    new_hexagon_vertices = hexagon_star_vertices((center_x, center_y), new_size)
                    cv2.fillPoly(snow_image, [new_hexagon_vertices], (255, 255, 255))

            # Blend the original image with the snowflake layer
            blended_image = cv2.addWeighted(original_image, 1, blurred_snow_image, 0.7, 0)
            blended_images.append(blended_image)

        return blended_images

        # Call add_snowflakes function to add snowflakes
        final_image = add_snowflakes(image)

        return final_image

# Set input and output folder paths
input_folder_path = '/content/drive/MyDrive/sensing_project/data/row_image/moderate'
output_folder_path = '/content/drive/MyDrive/sensing_project/data/augumented_snowfall_2/moderate'

# Ensure the output folder exists, if not, create it
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Get all files in the input folder
input_files = os.listdir(input_folder_path)

# Iterate through each file
for input_file_name in input_files:
    input_file_path = os.path.join(input_folder_path, input_file_name)

    # Read the image
    original_image = cv2.imread(input_file_path)

    # Process the image
    blended_image = generate_snow(original_image)

    # Generate the output file path
    output_file_path = os.path.join(output_folder_path, input_file_name)

    # Save the processed image
    cv2.imwrite(output_file_path, blended_image)

print("Processing complete, images saved to", output_folder_path)


处理完成，图像已保存到 /content/drive/MyDrive/sensing_project/data/augumented_snowfall_2/moderate
